In [15]:
#|default_exp arena_agent



In [16]:
# export
"""
Arena-Buddy web chat + LLM agent
--------------------------------

*   Re-uses every MCP tool exported in **03_fastmcp_tools.ipynb**
*   Uses the **agents** framework (same as *ONE.py*)
*   UI built with **FastHTML + HTMX + Tailwind/daisyUI**
"""

# %%
from __future__ import annotations
import os, asyncio, time, html
from pathlib import Path
from typing import AsyncIterator, List, Dict

from fasthtml import FastHTML         
from fasthtml.common import Div, Form, Input, Button, H1

from starlette.requests import Request
from starlette.responses import HTMLResponse, StreamingResponse

from agents import Agent, Runner
from agents.model_settings import ModelSettings
from agents.mcp import MCPServerSse          # thin wrapper for FastMCP SSE

# ────────────────────────────────────────────────────────────────────────────
# MCP endpoint of your tool-server (03_fastmcp_tools)
# ────────────────────────────────────────────────────────────────────────────
MCP_HOST   = os.getenv("MCP_HOST", "localhost")
MCP_PORT   = int(os.getenv("MCP_PORT", "9001"))
MCP_URL    = f"http://{MCP_HOST}:{MCP_PORT}/sse"

# ────────────────────────────────────────────────────────────────────────────
# FastHTML single-page scaffold
# ────────────────────────────────────────────────────────────────────────────
app = FastHTML(
    hdrs = (
        # Tailwind + daisyUI + HTMX-SSE extension
        '<script src="https://cdn.tailwindcss.com"></script>',
        '<script src="https://cdn.jsdelivr.net/npm/daisyui@4/dist/full.min.js"></script>',
        '<script src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"></script>',
    ),
    live=True,
)

MSG: List[Dict[str,str]] = []                       # in-memory chat history

def _chat_bubble(idx: int, **hx):
    who, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side    = "chat-end" if who == "user" else "chat-start"
    bubble  = "bg-sky-700 text-white" if who == "assistant" else "bg-gray-200"
    return Div(
        Div(who, cls="chat-header text-xs text-gray-500"),
        Div(txt if who=="user" else html.unescape(txt),
            cls=f"chat-bubble {bubble}", **hx),
        cls=f"chat {side}", id=f"m{idx}"
    )

def _chat_input():
    return Input(
        id="msgin", name="msg", type="text", autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full", hx_swap_oob="true",
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )

@app.route("/")
def home():
    return Div(
        H1("Arena Buddy", cls="text-3xl font-bold mb-4"),
        # chat log
        Div(id="chatlog", cls="space-y-3 mb-4 h-[70vh] overflow-y-auto bg-base-200 p-4 rounded-box"),
        # input-form (HTMX posts back)
        Form(
            Div(_chat_input(),
                Button("Send ✈", cls="btn btn-primary ml-2"),
                cls="flex"),
            hx_post="/send", hx_target="#chatlog", hx_swap="beforeend",
        ),
        cls="max-w-2xl mx-auto p-6",
    )

# ────────────────────────────────────────────────────────────────────────────
# LLM agent glue
# ────────────────────────────────────────────────────────────────────────────
async def _assistant_html(prompt: str) -> str:
    """
    Run the LLM agent and return **already-HTML-escaped** reply text.
    The agent has full access to every MCP tool – you DO NOT hard-code jokes,
    images, weather cards, etc.  Those will be produced on-the-fly by the LLM
    through tool calls.
    """
    async with MCPServerSse(name="ui", params={"url": MCP_URL}) as srv:
        agent = Agent(
            "assistant",
            # The style prompt: persuasion + humour + green-nudging
            instructions=(
                "You are Arena Buddy: convince the human to reach Metro Areena Espoo "
                "with the **lowest-emission** mode possible "
                "(walk 🚶, bike 🚴, public-transport 🚇, car 🚗 last-resort).  "
                "Retrieve live routes, weather, parking util etc. via MCP tools.  "
                "Embed fun/toy-size images or cards when helpful.  "
                "Respond in short friendly paragraphs."
            ),
            mcp_servers=[srv],
            model_settings=ModelSettings(tool_choice="auto")   # let LLM pick tools
        )
        result = await Runner.run(starting_agent=agent, input=prompt)
        return html.escape(result.final_output, quote=False)

# ────────────────────────────────────────────────────────────────────────────
# HTMX chat endpoint  (streams assistant reply over SSE)
# ────────────────────────────────────────────────────────────────────────────
@app.post("/send")
async def _send(request: Request):
    form  = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt: return HTMLResponse("")      # ignore empty

    # store 2 placeholders
    MSG.append({"role": "user",       "content": html.escape(prompt)})
    MSG.append({"role": "assistant",  "content": ""})
    user_idx, asst_idx = len(MSG)-2, len(MSG)-1

    # build fragments: user bubble + empty assistant bubble (SSE-enabled)
    user_html  = _chat_bubble(user_idx).__html__()
    asst_html  = _chat_bubble(
        asst_idx,
        hx_ext="sse",
        sse_connect=f"/stream/{asst_idx}",
        sse_swap="message",
        sse_close="close",
        hx_swap="innerHTML",
    ).__html__()

    return HTMLResponse(user_html + asst_html + _chat_input().__html__())

@app.get("/stream/{idx}")
async def _stream(idx: int):
    async def gen() -> AsyncIterator[str]:
        reply = await _assistant_html(MSG[idx-1]["content"])
        MSG[idx]["content"] = reply
        # simple SSE block
        yield f"event: message\ndata: {reply}\n\n"
        yield "event: close\ndata:\n\n"
    return StreamingResponse(gen(), media_type="text/event-stream")

# ────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    import uvicorn, logging; logging.basicConfig(level="INFO")
    # NB: FastMCP already runs in 03_fastmcp_tools; no need to start again.
    uvicorn.run(app, host="0.0.0.0", port=8000)


AttributeError: module 'tensorflow' has no attribute 'contrib'